## Group comparision analysis example for VRE. 
Example using TCGA ESCA (Easophageal cancer) obtained from Broad Inst TCGA data repo Firebrowser (<http://firebrowse.org>)

### Assumptions for analysis 

* User knows programming to be able to clean data and put it in the right format
* Two input data files: Gene expression data from RNA-seq (raw counts), and clinical data
* RNA-seq file format : Rows are features (gene names), Columns are samples. Only tumor samples were chosen for this analysis
* RNA-seq file format : Features are assumed to be in this format `GeneName|GeneID` . Example shown here is from public TCGA data obtained from Firebrowser. 
* Clinical data: This is a cleaned file that contains two columns - patient ids and vital status (Dead/Alive)
* User knows to identify baseline and comparison groups for analysis

## Goal of this analysis
* Use the gene expression data from RNA-seq (raw counts) obtained from tumor tissue of Esophageal cancer patients to compare Dead and Alive patients.
* Baseline Group (less screwed up group) = Alive
* Comparison Group (screwed up group) = Dead

## Analysis Steps

* Read in gene exp data
* Read in clinical data file
* Use the clinical data to separate gene expression data for baseline and comparison groups
* Set labels for the groups
* Call function to perform the group comparison analysis. This function will use the Bioconducor package `EdgeR` <https://bioconductor.org/packages/release/bioc/html/edgeR.html> . The package requires the RNA-seq data to be in the form of raw counts.
* Read in the results of group comp analysis 
* Select threshold for short listing results. Ideal threshold would result in number of genes  less than 700 or 1000.
* Call function to perform Enrichment Analysis. This is done using the EnrichR R package. <https://cran.r-project.org/web/packages/enrichR/index.html> 

### Step 0 - Before we start
* Use terminal (shell) to copy the input files from the storage bucket to the `/home/jupyter/tutorials/storage/input` folder. 
* Upload code files to the `/home/jupyter/tutorials/storage/code` folder
* Below is an example of how to use the R API package to make a local copy of the input files from google storage. But it can only copy one file at a time

In [2]:
# Example code
#makes local copy of the csv file
library(googleCloudStorageR)

#setting JSON authorization 
gcs_auth("/home/jupyter/tutorials/jsonauth/vre-pipeline-prototype-75d9d4263e97.json") 

#download object from google bucket -- this is a test example
gcs_get_object(object_name = "TCGA_ESCA_ClinDataVitalStatus.tsv", 
                        bucket = "vre-use-case-rna-seq-input", 
               saveToDisk = "/home/jupyter/tutorials/storage/input/TCGA_ESCA_ClinDataVitalStatus2.tsv", 
               overwrite = TRUE)

2021-04-14 20:24:39 -- Saved TCGA_ESCA_ClinDataVitalStatus.tsv to /home/jupyter/tutorials/storage/input/TCGA_ESCA_ClinDataVitalStatus2.tsv (4 Kb)



[1] TRUE

### Install packages 

In [46]:
#Install packages
install.packages("openxlsx")
library(stringr)

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("edgeR")
library(edgeR)
library(openxlsx)

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cloud.r-project.org


Bioconductor version 3.10 (BiocManager 1.30.12), R 3.6.3 (2020-02-29)

Installing package(s) 'edgeR'

Installation paths not writeable, unable to update packages
  path: /usr/lib/R/library
  packages:
    boot, class, cluster, codetools, KernSmooth, lattice, MASS, Matrix, mgcv,
    nlme, nnet, spatial, survival
  path: /usr/local/lib/R/site-library
  packages:
    broom, cli, cpp11, dbplyr, devtools, gargle, gert, gh, googleAuthR, keras,
    packrat, pillar, pkgload, processx, rbibutils, remotes, reprex, tinytex,
    vctrs, viridisLite



### Location of input files

In [24]:
enrichRFileLocation <- "/home/jupyter/tutorials/storage/input/20201203-EnrichR-Databases.txt"

geneExpLocation <- "/home/jupyter/tutorials/storage/input/20210318_TCGA_ESCA_RNAseq_GeneExp_RawCounts_Tumor.tsv"

clinFileLocation <- "/home/jupyter/tutorials/storage/input/TCGA_ESCA_ClinDataVitalStatus.tsv"

groupCompOutputLocation <- "/home/jupyter/tutorials/storage/output/edgeR_ExactTest.csv"

### ** Part 1 - Group comparison analysis **

#### Read in gene expression file

In [10]:
#20531 rows features, 173 tumor samples columns
geneExp <- read.csv(file = geneExpLocation, 
                    sep="\t", stringsAsFactors = F, row.names = 1)

# First 20 rows - no gene names
head(geneExp[1:20, 1:4])

#View Rows 5000-5010 - can see gene names
head(geneExp[5000:5010,1:3])

,TCGA.2H.A9GF,TCGA.2H.A9GG,TCGA.2H.A9GH,TCGA.2H.A9GI
,<dbl>,<dbl>,<dbl>,<dbl>
?|100130426,1.05,0.00,1.02,0.00
?|100133144,74.63,54.07,51.03,46.24
?|100134869,45.37,25.93,62.97,8.76
?|10357,165.00,70.00,201.98,111.71
?|10431,2158.00,2061.00,2078.00,1455.00
?|136542,0.00,0.00,0.00,0.00


,TCGA.2H.A9GF,TCGA.2H.A9GG,TCGA.2H.A9GH
,<dbl>,<dbl>,<dbl>
DLGAP2|9228,3,11,10
DLGAP3|58512,32,943,23
DLGAP4|22839,7327,8041,5178
DLGAP5|9787,2284,650,2744
DLK1|8788,1,0,0
DLK2|65989,7,175,8


#### Read in cleaned clinical data

In [11]:
## Read in cleaned clinical data

# 185 sample  IDs
clinData <- read.csv(file = clinFileLocation,
                     sep = "\t", stringsAsFactors = F)

head(clinData)


,V1,V2
,<chr>,<chr>
1,TCGA.IC.A6RF,alive
2,TCGA.JY.A6FB,alive
3,TCGA.JY.A938,alive
4,TCGA.L5.A43I,dead
5,TCGA.L5.A43J,dead
6,TCGA.L5.A43M,alive


#### Subset clinical data  into two data frames - dead and alive

In [12]:
## Subset clinical data patients into two data frames

clinDataAlive <- subset(clinData, V2 == "alive") #128 patients
clinDataDead <- subset(clinData, V2 == "dead") #57 patients dead


In [13]:
#checks
dim(clinDataAlive)

[1] 128   2

#### Subset gene exp data  into two data frames - dead and alive

In [14]:
## Subset gene exp data into two data frames - alive and dead
#Make sure data is numerical format, not strings

#Subset alive patients
matchBaseline <- which(colnames(geneExp) %in% clinDataAlive$V1)
geneExpBaseline <- geneExp[,matchBaseline] #124 samples alive

#Subset dead patients
matchComparison <- which(colnames(geneExp) %in% clinDataDead$V1)
geneExpComparison<- geneExp[,matchComparison] # 49 samples dead

#Get number of columns
nColBaseline <- ncol(geneExpBaseline)
nColComparision <- ncol(geneExpComparison)

In [16]:
#checks
dim(geneExpBaseline)
dim(geneExpComparison)

[1] 20531   124

[1] 20531    49

#### Prep data in the right format for the function for Group Comp

In [17]:
## Prep data to call function for Group Comp

# RNA-seq data for function
inputForGroupComp <- cbind(geneExpBaseline, geneExpComparison)

labelsForGroupComp <- c(rep("alive", nColBaseline),
                          rep("dead", nColComparision))

In [19]:
#checks
dim(inputForGroupComp)
head(labelsForGroupComp)

[1] 20531   173

[1] "alive" "alive" "alive" "alive" "alive" "alive"

#### Data is now prepped. Call function to run Group comp analysis using EdgeR package

In [20]:
## call function to run EdgeR
source("/home/jupyter/tutorials/storage/code/funcSubsetForEdgeR.R")


Loading required package: limma



In [22]:
## call function to run EdgeR
funcSubsetForEdgeR(inputData = inputForGroupComp, 
           groupLabels = labelsForGroupComp, 
           baselineGrp = "alive", 
           compGrp = "dead", 
           outputFileLocation = "/home/jupyter/tutorials/storage/output/edgeR")

Called from: funcSubsetForEdgeR(inputData = inputForGroupComp, groupLabels = labelsForGroupComp, 
    baselineGrp = "alive", compGrp = "dead", outputFileLocation = "/home/jupyter/tutorials/storage/output/edgeR")
debug: inputData[inputData < 0] = 0
debug: group_subset <- factor(as.character(groupLabels))
debug: group_subset <<- relevel(group_subset, ref = baselineGrp)
debug: if (min(inputData) < 0) {
    input2_Subset <<- inputData + 2
} else {
    input2_Subset <<- inputData
}
debug: input2_Subset <<- inputData
debug: funcEdgeR(inputData = input2_Subset, grpData = group_subset, 
    outFile = outputFileLocation)


#### Now the group comparison analysis is over. 
### ** Part 2 ** Read in the results of the group comp analysis. Set 4 different threshold options **

In [25]:
## Read in EdgeR output. Set thresholds. Get gene names. Remove duplicates

#17962 features. Make sure the data frame is a numeric data frame
edgeRoutput <- read.csv(file = groupCompOutputLocation, 
                  header = T, stringsAsFactors = F, row.names = 1)

### Set threshold values
pValueCutOff <- 0.01
logFCCutOff <- 1


#### Option 1 - find out how many features with p-value < 0.01

In [31]:
### Option 1 - find out how many features with p-value < 0.01

which1 <- which(as.numeric(edgeRoutput$PValue) <= pValueCutOff)
print("Option 1: Features with p-value < 0.01:")
print(length(which1))
option1 <- edgeRoutput[which1, ] #save results from Option 1


[1] "Option 1: Features with p-value < 0.01:"
[1] 6366


#### Option 2 - find out how many features with p-value < 0.01 and logFoldChange >= 1 or <= -1

In [27]:
### Option 2 - find out how many features with p-value < 0.01 and logFoldChange >= 1 or <= -1

which2 <- which((edgeRoutput$PValue <= pValueCutOff) & 
                    (edgeRoutput$logFC >= logFCCutOff | edgeRoutput$logFC <= -logFCCutOff))
print("Option 2: features with p-value < 0.01 and Log Fold Change >=1 or <= -1:")
print(length(which2))
option2 <- edgeRoutput[which2, ] #save results from Option 2

[1] "Option 2: features with p-value < 0.01 and Log Fold Change >=1 or <= -1:"
[1] 1274


#### Option 3 - find out how many features with FDR < 0.01

In [28]:
### Option 3 - find out how many features with FDR < 0.01

which3 <- which(as.numeric(edgeRoutput$FDR) <= 0.01)
print("Option 3: features with FDR < 0.01:")
print(length(which3))
option3 <- edgeRoutput[which3, ] #save results from Option 3

[1] "Option 3: features with FDR < 0.01:"
[1] 5598


#### Option 4 - find out how many features with FDR < 0.01 and LogFoldChange >= 1.5 or <= -1.5

In [29]:
### Option 4 - find out how many features with FDR < 0.01 and LogFoldChange >= 1.5 or <= -1.5

which4 <- which((edgeRoutput$FDR <= 0.01) & 
                    (edgeRoutput$logFC >= 1.5 | edgeRoutput$logFC <= -1.5))
print("Option 4: Features with FDR < 0.01, and Log Fold change >=1.5 or <= -1.5:")
print(length(which4))
option4 <- edgeRoutput[which4, ] #save results from Option 4

[1] "Option 4: Features with FDR < 0.01, and Log Fold change >=1.5 or <= -1.5:"
[1] 649


### ** Part 3 **
### Call function to run Enrichment analyasis (using EnrichR package) on the 4 options

In [50]:
####Calling function to subset gene results and run EnrichR for each option
source("/home/jupyter/tutorials/storage/code/funcSubsetForEnrichR.R")

# Option 1
library(openxlsx)
funcSubsetForEnrichR(shortListResults = option1, 
                     filename1 = "/home/jupyter/tutorials/storage/output/option1_shortlist_results.csv", 
                     optionName1 = "option1")

Uploading data to Enrichr... Done.
  Querying KEGG_2019_Human... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying WikiPathways_2019_Human... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying KEGG_2019_Mouse... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying Reactome_2016... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying BioPlanet_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying ClinVar_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying TRRUST_Transcription_Factors_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying Transcription_Factor_PPIs... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying TRANSFAC_and_JASPAR_PWMs... Done.
Parsing results

In [52]:
####Calling function to subset gene results and run EnrichR for 
# Option 2
library(openxlsx)
funcSubsetForEnrichR(shortListResults = option2, 
                     filename1 = "/home/jupyter/tutorials/storage/output/option2_shortlist_results.csv", 
                     optionName1 = "option2")

Uploading data to Enrichr... Done.
  Querying KEGG_2019_Human... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying WikiPathways_2019_Human... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying KEGG_2019_Mouse... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying Reactome_2016... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying BioPlanet_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying ClinVar_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying TRRUST_Transcription_Factors_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying Transcription_Factor_PPIs... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying TRANSFAC_and_JASPAR_PWMs... Done.
Parsing results

In [47]:
####Calling function to subset gene results and run EnrichR for 
# Option 3
library(openxlsx)
funcSubsetForEnrichR(shortListResults = option3, 
                     filename1 = "/home/jupyter/tutorials/storage/output/option3_shortlist_results.csv", 
                     optionName1 = "option3")

Uploading data to Enrichr... Done.
  Querying KEGG_2019_Human... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying WikiPathways_2019_Human... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying KEGG_2019_Mouse... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying Reactome_2016... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying BioPlanet_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying ClinVar_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying TRRUST_Transcription_Factors_2019... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying Transcription_Factor_PPIs... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying TRANSFAC_and_JASPAR_PWMs... Done.
Parsing results

In [ ]:
####Calling function to subset gene results and run EnrichR for 
# Option 4
library(openxlsx)
funcSubsetForEnrichR(shortListResults = option4, 
                     filename1 = "/home/jupyter/tutorials/storage/output/option4_shortlist_results.csv", 
                     optionName1 = "option4")

#### All Analaysis steps done. 
### Part 4 - Copy output files from here to google storage

One way is to use R package: Reference document: https://cran.r-project.org/web/packages/googleCloudStorageR/vignettes/googleCloudStorageR.html 

**Easier to use terminal to copy entire "output" folder** 

In [54]:
gcs_upload(file = "/home/jupyter/tutorials/storage/output/option1_shortListedUniqueGenes.tsv", 
           bucket = "vre-use-case-rna-seq" , predefinedAcl = "bucketLevel")

2021-04-13 18:25:34 -- File size detected as 41 Kb



==Google Cloud Storage Object==
Name:                /home/jupyter/tutorials/storage/output/option1_shortListedUniqueGenes.tsv 
Type:                text/tab-separated-values 
Size:                41 Kb 
Media URL:           https://www.googleapis.com/download/storage/v1/b/vre-use-case-rna-seq/o/%2Fhome%2Fjupyter%2Ftutorials%2Fstorage%2Foutput%2Foption1_shortListedUniqueGenes.tsv?generation=1618338334327402&alt=media 
Download URL:        https://storage.cloud.google.com/vre-use-case-rna-seq/%2Fhome%2Fjupyter%2Ftutorials%2Fstorage%2Foutput%2Foption1_shortListedUniqueGenes.tsv 
Public Download URL: https://storage.googleapis.com/vre-use-case-rna-seq/%2Fhome%2Fjupyter%2Ftutorials%2Fstorage%2Foutput%2Foption1_shortListedUniqueGenes.tsv 
Bucket:              vre-use-case-rna-seq 
ID:                  vre-use-case-rna-seq//home/jupyter/tutorials/storage/output/option1_shortListedUniqueGenes.tsv/1618338334327402 
MD5 Hash:            WBs/yPBuR7p+EL7Q+rh3ig== 
Class:               STANDARD 
C

#### Run in terminal.

In [ ]:
# run in terminal - copy one file to google bucket
gsutil cp /home/jupyter/tutorials/storage/output/option2_shortListedUniqueGenes.tsv gs://vre-use-case-rna-seq

In [ ]:
# run in terminal. Copy folder to google bucket
gsutil -m cp -R /home/jupyter/tutorials/storage/output/ gs://vre-use-case-rna-seq